# Dark Pool Trade Analysis

This notebook connects to the production database and analyzes dark pool trades.

In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import pytz
from dotenv import load_dotenv

# Set plot style
#plt.style.use('seaborn')
#"sns.set_palette('deep')
%matplotlib inline

In [19]:
# Database connection setup
DB_CONFIG = {
    'dbname': 'defaultdb',
    'user': 'doadmin',
    'password': 'AVNS_SrG4Bo3B7uCNEPONkE4',
    'host': 'vvv-trading-db-do-user-2110609-0.i.db.ondigitalocean.com',
    'port': '25060'
}

# Create database URL
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# Create engine with SSL required
engine = create_engine(
    DATABASE_URL,
    connect_args={
        'sslmode': 'require'
    }
)

# Get Trades from spefiic Date

In [3]:
# Function to fetch trades for a specific date
def fetch_trades(date_str=None):
    query = """
    SELECT *
    FROM trading.darkpool_trades
    WHERE DATE(executed_at) = %(date)s
    ORDER BY executed_at
    """
    
    if date_str is None:
        date_str = '2025-04-17'  # Default to our test data date
    
    # Pass parameters as a dictionary
    params = {'date': date_str}
    return pd.read_sql_query(query, engine, params=params)

# Fetch trades for April 17th
trades_df = fetch_trades('2025-04-17')
print(f"Fetched {len(trades_df)} trades")
trades_df.head()

Fetched 15 trades


,id,tracking_id,symbol,size,price,volume,premium,executed_at,nbbo_ask,nbbo_bid,market_center,sale_cond_codes,collection_time,created_at
0,415,70331564740492,QQQ,493.0,445.3000,44810365.0,219532.9000,2025-04-17 23:32:11+00:00,445.55,445.18,L,None,2025-04-19 19:54:42.651306+00:00,2025-04-19 19:54:42.663577+00:00
1,414,70374608487124,SPY,473.0,527.2100,79846628.0,249370.3300,2025-04-17 23:32:54+00:00,527.24,527.01,L,None,2025-04-19 19:54:42.651306+00:00,2025-04-19 19:54:42.663577+00:00
2,413,70454350009635,QQQ,299.0,445.3884,44811967.0,133171.1316,2025-04-17 23:34:14+00:00,445.39,445.30,L,None,2025-04-19 19:54:42.651306+00:00,2025-04-19 19:54:42.663577+00:00
3,412,70469265910490,QQQ,1000.0,445.2500,44813023.0,445250.0000,2025-04-17 23:34:29+00:00,445.39,445.30,L,None,2025-04-19 19:54:42.651306+00:00,2025-04-19 19:54:42.663577+00:00
4,411,70629848116153,QQQ,729.0,445.1111,44814745.0,324485.9919,2025-04-17 23:37:09+00:00,445.40,445.11,L,None,2025-04-19 19:54:42.651306+00:00,2025-04-19 19:54:42.663577+00:00


# Get Latest Trades Log from Prod DB

In [41]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
DB_CONFIG = {
    'dbname': 'defaultdb',
    'user': 'doadmin',
    'password': 'AVNS_SrG4Bo3B7uCNEPONkE4',
    'host': 'vvv-trading-db-do-user-2110609-0.i.db.ondigitalocean.com',
    'port': '25060'
}

# Create database URL
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# Create engine with SSL required
engine = create_engine(
    DATABASE_URL,
    connect_args={
        'sslmode': 'require'
    }
)

# Query for logs and recent data collection status
query = """
WITH recent_data AS (
    -- Check darkpool trades
    SELECT 
        'Darkpool Trades' as collector,
        MAX(collection_time) as last_collection,
        COUNT(*) as records_last_hour
    FROM trading.darkpool_trades
    WHERE collection_time >= NOW() - INTERVAL '1 hour'
    
    UNION ALL
    
    -- Check options flow
    SELECT 
        'Options Flow' as collector,
        MAX(collected_at) as last_collection,
        COUNT(*) as records_last_hour
    FROM trading.options_flow
    WHERE collected_at >= NOW() - INTERVAL '1 hour'
),
recent_logs AS (
    SELECT 
        timestamp,
        level,
        message,
        date_trunc('minute', timestamp) as log_minute,
        count(*) over (partition by date_trunc('minute', timestamp)) as logs_per_minute
    FROM trading.collector_logs
    WHERE timestamp >= NOW() - INTERVAL '1 hour'
    ORDER BY timestamp DESC
    LIMIT 10
)

-- Combine results
SELECT 
    'Collection Status' as section,
    collector,
    last_collection,
    records_last_hour as "Records (Last Hour)"
FROM recent_data

UNION ALL

SELECT 
    'Recent Logs' as section,
    level as collector,
    timestamp as last_collection,
    logs_per_minute as "Records (Last Hour)"
FROM recent_logs
ORDER BY section, last_collection DESC
"""

results_df = pd.read_sql_query(query, engine)

# Display collection status
collection_status = results_df[results_df['section'] == 'Collection Status']
print("\nCollection Status:")
display(collection_status)

# Display recent logs
logs = results_df[results_df['section'] == 'Recent Logs']
print("\nMost recent log entries:")
display(logs)


Collection Status:


,section,collector,last_collection,Records (Last Hour)
0,Collection Status,Options Flow,NaT,0
1,Collection Status,Darkpool Trades,2025-04-30 18:35:02.892928+00:00,319



Most recent log entries:


,section,collector,last_collection,Records (Last Hour)
2,Recent Logs,INFO,2025-04-30 18:35:02.927984+00:00,16
3,Recent Logs,INFO,2025-04-30 18:35:02.925801+00:00,16
4,Recent Logs,INFO,2025-04-30 18:35:02.923132+00:00,16
5,Recent Logs,INFO,2025-04-30 18:35:02.906651+00:00,16
6,Recent Logs,INFO,2025-04-30 18:35:02.904138+00:00,16
7,Recent Logs,INFO,2025-04-30 18:35:02.898956+00:00,16
8,Recent Logs,INFO,2025-04-30 18:35:02.896599+00:00,16
9,Recent Logs,INFO,2025-04-30 18:35:02.890667+00:00,16
10,Recent Logs,INFO,2025-04-30 18:35:02.886052+00:00,16
11,Recent Logs,INFO,2025-04-30 18:35:02.881828+00:00,16


# Get All Trades and Save as CSV

In [18]:
import pandas as pd
from sqlalchemy import create_engine
import os
from datetime import datetime

# Database connection setup (same as before)
DB_CONFIG = {
    'dbname': 'defaultdb',
    'user': 'doadmin',
    'password': 'AVNS_SrG4Bo3B7uCNEPONkE4',
    'host': 'vvv-trading-db-do-user-2110609-0.i.db.ondigitalocean.com',
    'port': '25060'
}

DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
engine = create_engine(DATABASE_URL, connect_args={'sslmode': 'require'})

# Query for dark pool trades (same as before)
darkpool_query = """
SELECT 
    t.*,
    date_trunc('hour', t.executed_at) as trade_hour,
    t.price - t.nbbo_bid as price_impact,
    (t.price - t.nbbo_bid) / t.nbbo_bid as price_impact_pct,
    CASE 
        WHEN t.size >= 10000 THEN 'Block Trade'
        WHEN t.premium >= 0.02 THEN 'High Premium'
        ELSE 'Regular'
    END as trade_type,
    count(*) over (partition by t.symbol, date_trunc('hour', t.executed_at)) as trades_per_hour,
    sum(t.size) over (partition by t.symbol, date_trunc('hour', t.executed_at)) as volume_per_hour
FROM trading.darkpool_trades t
ORDER BY t.executed_at DESC
"""

# New query for options flow with similar metrics
options_query = """
SELECT 
    f.*,
    date_trunc('hour', f.collected_at) as flow_hour,
    CASE 
        WHEN f.premium >= 1000000 THEN 'Whale'
        WHEN f.premium >= 100000 THEN 'Large'
        ELSE 'Regular'
    END as flow_size,
    count(*) over (partition by f.symbol, date_trunc('hour', f.collected_at)) as flows_per_hour,
    sum(f.premium) over (partition by f.symbol, date_trunc('hour', f.collected_at)) as premium_per_hour,
    sum(f.contract_size) over (partition by f.symbol, date_trunc('hour', f.collected_at)) as contracts_per_hour
FROM trading.options_flow f
ORDER BY f.collected_at DESC
"""

# Fetch both datasets
print("Fetching all dark pool trades...")
trades_df = pd.read_sql_query(darkpool_query, engine)

print("Fetching all options flow data...")
options_df = pd.read_sql_query(options_query, engine)

# Process darkpool trades
trades_df['executed_at'] = pd.to_datetime(trades_df['executed_at'])
trades_df['collection_time'] = pd.to_datetime(trades_df['collection_time'])
trades_df['trade_hour'] = pd.to_datetime(trades_df['trade_hour'])

# Process options flow
options_df['collected_at'] = pd.to_datetime(options_df['collected_at'])
options_df['created_at'] = pd.to_datetime(options_df['created_at'])
options_df['expiry'] = pd.to_datetime(options_df['expiry'])
options_df['flow_hour'] = pd.to_datetime(options_df['flow_hour'])

# Create data directory
os.makedirs('data', exist_ok=True)

# Generate filenames with current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
darkpool_filename = f'data/darkpool_trades_all_{timestamp}.csv'
options_filename = f'data/options_flow_all_{timestamp}.csv'

# Save both datasets
trades_df.to_csv(darkpool_filename, index=False)
options_df.to_csv(options_filename, index=False)

print(f"\nSaved {len(trades_df)} trades to {darkpool_filename}")
print(f"Saved {len(options_df)} option flows to {options_filename}")

# Print darkpool trade summary
print("\nDarkpool Trade summary by symbol:")
print(trades_df.groupby('symbol').agg({
    'size': ['count', 'sum', 'mean'],
    'premium': ['mean', 'max'],
    'price_impact_pct': 'mean'
}).round(2))

# Print options flow summary
print("\nOptions Flow summary by symbol:")
print(options_df.groupby('symbol').agg({
    'premium': ['count', 'sum', 'mean', 'max'],
    'contract_size': ['sum', 'mean'],
    'iv_rank': 'mean'
}).round(2))

# Print date ranges for both datasets
print("\nDate ranges:")
print("Darkpool Trades:")
print(f"Earliest trade: {trades_df['executed_at'].min()}")
print(f"Latest trade: {trades_df['executed_at'].max()}")
print(f"Total trades: {len(trades_df)}")
print(f"Total volume: {trades_df['size'].sum():,.0f}")

print("\nOptions Flow:")
print(f"Earliest flow: {options_df['collected_at'].min()}")
print(f"Latest flow: {options_df['collected_at'].max()}")
print(f"Total flows: {len(options_df)}")
print(f"Total premium: ${options_df['premium'].sum():,.2f}")

Fetching all dark pool trades...
Fetching all options flow data...

Saved 15131 trades to data/darkpool_trades_all_20250502_210323.csv
Saved 0 option flows to data/options_flow_all_20250502_210323.csv

Darkpool Trade summary by symbol:
        size                        premium               price_impact_pct
       count        sum     mean       mean           max             mean
symbol                                                                    
EFA      172   643173.0  3739.38  315045.36  6.541874e+06              0.0
EZU       16    82671.0  5166.94  286552.86  7.372200e+05              0.0
FXF       11    23632.0  2148.36  231496.59  5.462500e+05              0.0
GLD     1384  1499634.0  1083.55  332387.21  2.323045e+07              0.0
QQQ     4310  4057818.0   941.49  435279.19  1.137197e+08              0.0
SLV      109   944937.0  8669.15  259586.57  1.963644e+06             -0.0
SMH      186   855799.0  4601.07  920931.08  6.960936e+07              0.0
SPY     8099  

In [79]:
# Simple query to check options_flow table
check_query = """
SELECT 
    COUNT(*) as total_records,
    MIN(collected_at) as earliest_record,
    MAX(collected_at) as latest_record,
    COUNT(DISTINCT symbol) as unique_symbols
FROM trading.options_flow;

-- Also show a sample of recent records
SELECT 
    symbol,
    strike,
    expiry,
    flow_type,
    premium,
    contract_size,
    iv_rank,
    collected_at
FROM trading.options_flow
ORDER BY collected_at DESC
LIMIT 5;
"""

# Run the query
results = pd.read_sql_query(check_query, engine)
display(results)

,symbol,strike,expiry,flow_type,premium,contract_size,iv_rank,collected_at


In [5]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
DB_CONFIG = {
    'dbname': 'defaultdb',
    'user': 'doadmin',
    'password': 'AVNS_SrG4Bo3B7uCNEPONkE4',
    'host': 'vvv-trading-db-do-user-2110609-0.i.db.ondigitalocean.com',
    'port': '25060'
}

# Create database URL
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# Create engine with SSL required
engine = create_engine(
    DATABASE_URL,
    connect_args={
        'sslmode': 'require'
    }
)

# First, let's check the table structure
table_info_query = """
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_schema = 'trading' 
AND table_name = 'options_flow' 
ORDER BY ordinal_position;
"""

print("Table Structure:")
table_info_df = pd.read_sql_query(table_info_query, engine)
display(table_info_df)

Table Structure:


,column_name,data_type
0,id,integer
1,symbol,character varying
2,strike,numeric
3,expiry,date
4,flow_type,character varying
5,premium,numeric
6,contract_size,integer
7,iv_rank,numeric
8,collected_at,timestamp with time zone
9,created_at,timestamp with time zone


In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
DB_CONFIG = {
    'dbname': 'defaultdb',
    'user': 'doadmin',
    'password': 'AVNS_SrG4Bo3B7uCNEPONkE4',
    'host': 'vvv-trading-db-do-user-2110609-0.i.db.ondigitalocean.com',
    'port': '25060'
}

# Create database URL
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# Create engine with SSL required
engine = create_engine(
    DATABASE_URL,
    connect_args={
        'sslmode': 'require'
    }
)

# Query 1: Hourly statistics
hourly_query = """
SELECT 
    date_trunc('hour', collected_at AT TIME ZONE 'UTC' AT TIME ZONE 'Europe/Copenhagen') as hour_cest,
    COUNT(*) as flow_count,
    COUNT(DISTINCT symbol) as unique_symbols,
    SUM(CASE WHEN flow_type = 'call' THEN 1 ELSE 0 END) as calls,
    SUM(CASE WHEN flow_type = 'put' THEN 1 ELSE 0 END) as puts,
    SUM(premium) as total_premium,
    MIN(collected_at AT TIME ZONE 'UTC' AT TIME ZONE 'Europe/Copenhagen') as first_signal,
    MAX(collected_at AT TIME ZONE 'UTC' AT TIME ZONE 'Europe/Copenhagen') as last_signal
FROM trading.options_flow
WHERE collected_at >= '2025-04-28'::date AT TIME ZONE 'Europe/Copenhagen'
    AND collected_at < '2025-04-29'::date AT TIME ZONE 'Europe/Copenhagen'
GROUP BY date_trunc('hour', collected_at AT TIME ZONE 'UTC' AT TIME ZONE 'Europe/Copenhagen')
ORDER BY hour_cest;
"""

# Query 2: Top symbols during 16:00-17:00 CEST
top_symbols_query = """
SELECT 
    symbol,
    COUNT(*) as flow_count,
    SUM(CASE WHEN flow_type = 'call' THEN 1 ELSE 0 END) as calls,
    SUM(CASE WHEN flow_type = 'put' THEN 1 ELSE 0 END) as puts,
    SUM(premium) as total_premium,
    AVG(delta) as avg_delta,
    AVG(iv_rank) as avg_iv_rank
FROM trading.options_flow
WHERE collected_at >= '2025-04-28 16:00:00'::timestamp AT TIME ZONE 'Europe/Copenhagen'
    AND collected_at < '2025-04-28 17:00:00'::timestamp AT TIME ZONE 'Europe/Copenhagen'
GROUP BY symbol
ORDER BY flow_count DESC
LIMIT 5;
"""

# Query 3: Error logs
error_logs_query = """
SELECT 
    timestamp AT TIME ZONE 'UTC' AT TIME ZONE 'Europe/Copenhagen' as timestamp_cest,
    level,
    message
FROM trading.collector_logs
WHERE timestamp >= '2025-04-28'::date AT TIME ZONE 'Europe/Copenhagen'
    AND timestamp < '2025-04-29'::date AT TIME ZONE 'Europe/Copenhagen'
    AND level IN ('ERROR', 'WARNING')
ORDER BY timestamp DESC;
"""

# Execute queries and display results
print("\nHourly Statistics:")
hourly_df = pd.read_sql_query(hourly_query, engine)
display(hourly_df)

print("\nTop Symbols 16:00-17:00 CEST:")
top_symbols_df = pd.read_sql_query(top_symbols_query, engine)
display(top_symbols_df)

print("\nError Logs:")
error_logs_df = pd.read_sql_query(error_logs_query, engine)
display(error_logs_df)


Hourly Statistics:


,hour_cest,flow_count,unique_symbols,calls,puts,total_premium,first_signal,last_signal



Top Symbols 16:00-17:00 CEST:


,symbol,flow_count,calls,puts,total_premium,avg_delta,avg_iv_rank



Error Logs:


,timestamp_cest,level,message
0,2025-04-28 17:55:02.727861+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
1,2025-04-28 17:50:02.403742+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
2,2025-04-28 17:45:02.940508+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
3,2025-04-28 17:40:02.605169+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
4,2025-04-28 17:35:02.215968+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
5,2025-04-28 17:30:02.752612+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
6,2025-04-28 17:25:02.333510+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
7,2025-04-28 17:20:03.085248+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
8,2025-04-28 17:15:02.719436+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
9,2025-04-28 17:10:02.324190+00:00,ERROR,Request failed: 429 Client Error: Too Many Req...
